# 新聞爬蟲 –– BBC新聞網

### 1. 用 selenium 擷取瀑布式網頁

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import random
import datetime

In [23]:
chromedriver = '/Users/daphne/JupyterProjects/ETF/新聞爬蟲/chromedriver3'
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.bbc.co.uk/search?q=us+china+trade")

#find click button path
more_path = '//*[@id= "search-content"]/nav[1]/a'
'''//*[@id="search-content"]/nav[1]/a'''

for i in range(35):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element_by_xpath(more_path).click()
    driver.implicitly_wait(5)

pageSource = driver.page_source
bsObj = BeautifulSoup(pageSource)

bsObj

<html class="b-reith-sans-font b-footer--transparent--dark-grey orb-js id-svg b-reith-sans-loaded js hashchange history cssanimations csstransitions video audio localstorage svg no-windows loaded se-no-touch orb-more-loaded" lang="en-GB"><head> <title>BBC - Search results for us china trade</title> <meta content="BBC, Search" name="keywords"/> <meta content="Application" name="description"/> <meta content="text/html; charset=utf-8" http-equiv="content-type"/> <meta content="width=device-width, initial-scale=1.0" name="viewport"/> <!-- Nav Env: live -->
<!-- Analytics Web Module: 0.0.1-301.8550366 -->
<!-- NavID Web Module: 0.2.0-65.de8f099 -->
<!-- Detectview Web Module: 0.0.1-301.8550366 -->
<!-- Searchbox Web Module: 3.0.0-24.9a69f1b -->
<!-- Promo Web Module: 3.0.0-137.2403d38 -->
<meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="100004154058350" property="fb:admins"/><link href="https://nav.files.bbci.co.uk/orbit/2.0.0-580.7ecee166/css/orb-ltr.mi

### 2. 抓取新聞 date, title, summary, content

In [24]:
#nodes = bsObj.find_all('article', {'class':"has_image media-video"})
nodes = bsObj.find_all('article')

print(len(nodes))
print(nodes)

350
[<article class="has_image media-text" data-bbc-container="search-query-results" data-bbc-metadata='{"PAR":"search-cards::1","CHD":1,"IMG":"news\/384\/cpsprodpb\/17964\/production\/_107221669_gettyimages-871917758-2.jpg"}' data-bbc-personalisation='{"EDN":"default","QRY":"us china trade"}' data-bbc-source="News | Business" data-bbc-title="US says China playing 'blame game' in trade battle" itemscope="">
<aside class="flags top">
<dl>
<dt>Published Date</dt>
<dd>
<time class="display-date" datetime="2019-06-04T02:39:20Z">
           4 Jun 2019       </time>
</dd>
</dl>
</aside>
<a href="https://www.bbc.co.uk/news/business-48507612" id="search-result-0">
<picture>
<img alt="" class="" src="https://ichef.bbci.co.uk/news/384/cpsprodpb/17964/production/_107221669_gettyimages-871917758-2.jpg" srcset="https://ichef.bbci.co.uk/news/384/cpsprodpb/17964/production/_107221669_gettyimages-871917758-2.jpg"/> </picture>
</a>
<aside class="flags mid">
<dl>
<dt>Published Date</dt>
<dd>
<time class

In [1]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
proxies = ""

# get content of news
def get_doc(href):
    resp = requests.get(href, headers=headers, proxies=proxies)
    resp.encoding = 'utf-8'
    bsObj_news = BeautifulSoup(resp.content, "lxml")
    a_news = bsObj_news.find_all('p', class_="")
    docs = []
    for d in a_news:
        doc = d.get_text()
        docs.append(doc)
    
    a_doc = ''
    for doc in docs:
        all_doc = a_doc + doc
        a_doc = all_doc 
    
    return a_doc


In [26]:
news = []
for node in nodes:
    title = node.find('h1', {'itemprop': "headline"}).get_text()
    summary = node.find('p', {'class': "summary long"}).get_text()
    href = node.find('a').get('href')
    doc = get_doc(href)
    try:
        time = node.find('time', {'class': "display-date"}).get_text().strip()
        #print(type(time))  #str
    except AttributeError:
        time = None
    #href = node.find({'a': 'href'})
    #print(time)
    #print(title)
    #print(href)
    #print(summary)
    #print(doc)
    #print("\n")
    if time != None:    # 避開非新聞的文章   # len=346
        node_dict = dict()
        node_dict['time'] = time
        node_dict['title'] = title
        node_dict['summary'] = summary
        node_dict['doc'] = doc
        news.append(node_dict)
    else:
        pass

#print(len(nodes))   #350

4 Jun 2019
US says China playing 'blame game' in trade battle
In a statement, the Trump administration also accused China of "backpedalling" on trade agreements.The comments come in response to Beijing's release of a paper blaming Washington for the setback in talks.The US reignited the trade war last month by raising tariffs on billions of dollars worth of Chinese goods, prompting Beijing to retaliate. The two countries have been in an escalating conflict over trade for the past year. The scope of the battle has expanded in recent months as Washington has tightened trade restrictions on Chinese telecoms giant Huawei. Hopes for an imminent trade deal were shattered in May after the Trump administration more than doubled tariffs on $200bn (£157.9bn) of Chinese imports and threatened additional duties."Our negotiating positions have been consistent throughout these talks, and China backpedalled on important elements of what the parties had agreed to," a statement from the Office of the U

22 May 2019
US China trade war: What has Trump got to do with trainers?



4 Jun 2019
World Bank warns of weaker global growth
The bank said it now expects growth of 2.6% for 2019 edging up to 2.7% the following year. The slowdown is widespread, according to the Bank's economists, affecting many countries.And there are risks to even this subdued outlook, including the uncertainty for business created by international trade tensions. One of the lead authors of the report, Franziska Ohnsorge, said the World Bank had warned in its previous forecasts six months ago of darkening skies."Then it was a forecast" she told the BBC. "But now we see it in the data."In January, the World Bank changed its outlook for 2019 from 3% growth to 2.9%. There has been a "broad based disappointment" affecting trade, investment and manufacturing and developed as well as emerging economies, Ms Ohnsorge said.Trade conflict has been an important factor behind the weaker growth, in particular the tension between 

11 May 2019
Weekend: US-China trade talks 'constructive'
The US Treasury Secretary, Steven Mnuchin, has described negotiations with China to try to resolve the two countries' trade war as 'constructive'.The US Treasury Secretary, Steven Mnuchin, has described negotiations with China to try to resolve the two countries' trade war as 'constructive'. So is a deal nearly done?Also, nearly all countries agree to stem the flow of plastic waste into poor nations.Joining Paul Henley to discuss these and other stories are Patrick Basham, director of the Washington-based think tank The Democracy Institute, and Ulrike Frenke, a policy fellow at the European Council on Foreign Relations.(Image: Signs with the US flag and Chinese flag are seen at the Qingdao free trade port area in Qingdao in China's eastern Shandong province. Credit: STR/AFP/Getty Images)

            See all episodes from Weekend
        



15 May 2019
Trump ropes Federal Reserve into US-China trade war
The trade dispute between

10 May 2019
US-China trade war: Your questions answered
The US has more than doubled tariffs on many Chinese goods, in an escalation of the tensions between the countries.Here are some of the questions you asked:There's no obvious direct impact on European consumers but the conflict is seen as a significant threat to global economic growth. For one thing, these are the two largest economies in the world and very important markets for many other countries.If the tariffs weaken them, it means less demand for goods from the rest of the world, Europe included. That could affect European consumers' incomes.The trade war has weighed on the global economy over the past year and created uncertainty for businesses and consumers.There is also the possibility of goods being diverted on to global markets, producing some downward pressure on prices in Europe and elsewhere.  A trade deficit is the difference between how much a country imports from other countries and how much it exports.Yes, China h

10 May 2019
News Daily: US-China trade war escalates, and English football makes European history
If you want to get this briefing by email, sign-up hereThe ongoing US-China trade war appears to be escalating, with the US more than doubling tariffs on $200bn (£150bn) worth of Chinese products. China says it "deeply regrets" the move - which comes as officials from both sides try to salvage a trade deal - and is threatening "necessary counter-measures". The International Monetary Fund's chief economist has previously blamed tensions between the two nations for contributing to a global economic slowdown.You can see which products are affected - and how the dispute has affected stock markets - in our series of charts. If you're struggling to get your head around what it all means, take a look at our 300-word explainer. And for a reminder of the knock-on effects for UK and global businesses and consumers, check out our report on the trade war's early victims."Brightening" clouds to refreez

...

17 Oct 2018
US moves to negotiate trade deals with Japan, UK, EU
It could take several months before the negotiations begin, it said.The move is part of US President Donald Trump's efforts to reshape trade policies to support his "America First" agenda. It comes as the US has been fighting a trade war with China, the world's second largest economy. "We will continue to expand U.S. trade and investment by negotiating trade agreements with Japan, the EU and the United Kingdom," US trade representative Robert Lighthizer said."We are committed to concluding these negotiations with timely and substantive results for American workers, farmers, ranchers, and businesses."The US plans to start negotiations with Japan, the world's third largest economy, "as soon as practicable, but no earlier than 90 days from the date of this notice," Mr Lighthizer wrote in a letter to Congress.Negotiations with the United Kingdom will start "as soon as it is ready" after it leaves the European Union in March 2

26 Apr 2018
Business Matters: Facebook Reports Surge in Revenue Amid Data Crisis
The site's revenue rose to $11.97 billion and active monthly users topped 2.2 billion.The site's revenue rose to $11.97 billion and active monthly users topped 2.2 billion. We ask Russell Brandom, senior reporter at The Verge, why Facebook doesn't appear to have suffered from the data privacy scandal. We speak to our correspondent Robin Brant from the Beijing Motor Show about how the US-China trade spat could impact BMW. Also, when it comes to philanthropy who gives money and why? Anna-Marie Harling, UBS Head of the Global Ultra High Net Worth Philanthropy Centre tells us about what the data shows. Plus, Grace Cathedral in San Francisco is holding a Beyonce Mass it says celebrates black female spirituality. The Reverend Yolanda Norton who is holding the mass explains the reason behind it.Plus we're joined throughout the programme by Peter Morici, Professor of International Business at the University of Mar

26 Jul 2018
Qualcomm axes NXP deal after failing to get Chinese approval
A deadline for the transaction, which needed China's sign-off, was set for 11:59pm (03:59 GMT) New York time under the agreement.The deal had already won the necessary regulatory approvals in eight of nine countries. It would have been one of the biggest deals between technology companies.It comes as US-China trade tensions have escalated, prompting speculation this latest development is a retaliation to US tariffs."We intend to terminate our purchase agreement to acquire NXP when the agreement expires at the end of the day today, pending any new material developments,"  said Steve Mollenkopf, CEO of Qualcomm, in the company's third quarter fiscal results.Qualcomm needed China's approval because it accounted for nearly two-thirds of its revenue last year, according to Reuters.The latest results showed revenues jumping 6% to $5.6bn.US President Donald Trump recently ramped up a trade war with China, threatening tar

3 Dec 2018
Best of Today: Monday's business with Dominic O'Connell
The US and China have reached a temporary truce in their trade war - but can they achieve a lasting deal?(Image: US President Donald Trump (R) and China"s President Xi Jinping (L) Credit: AFP/Getty Images)

            See all episodes from Best of Today
        



25 Oct 2018
Europe markets calm after Asia and US sink
Tokyo stocks slumped more than 3%, while losses pushed the US Dow Jones Industrial Average and the S&P 500 into negative territory for the year.European markets followed suit at first, but had risen by lunchtime, with London's FTSE 100 up 0.17%.Concerns over corporate profits and slowing growth have rattled investors.In the US, the Dow Jones Industrial Average sank 2.4% to 24,583.4 points, while the S&P 500 plunged 3.1% to 2,656.1 points on Wednesday. The technology-focused Nasdaq dropped more than 4.4% to 7,108.4 points - its worst day since 2011 - amid concerns about weak corporate profits and global t

3 Mar 1999
US China trade negotiations underway
The United States Trade Representative Charlene Barshefsky is in Beijing for the latest round of negotiations over China's bid to enter the World Trade Organisation.
The US, which has a fifty-billion dollar trade deficit with Beijing, has blocked China's previous attempts to join the world trade body and has called for it to allow foreign companies greater access to its markets. China is concerned that opening its markets will lead to increased unemployment. Before meeting the Chinese foreign trade minister Shi Guangsheng, Miss Barshefsky said she was ready to listen to Chinese proposals.
On Monday, the American Secretary of State, Madeleine Albright, said the United States would not link China's bid to join the World Trade Organisation with progress on human rights.
From the newsroom of the BBC World Service





Back to top | BBC News Home | BBC Homepage | ©




	var si = document.location+"";
	var tsi = si.replace(".stm","").substr(si.

In [2]:
#print(len(news)) #346(omitted 'time = None' article)

### 3. 將資料存成 .pkl

In [1]:
#存取資料
import pickle
fh = open('bbc_tradewar', 'wb')
pickle.dump(news, fh)
fh.close()